# Image dataset

- Official Croissant example of MNIST: https://github.com/mlcommons/croissant/tree/main/datasets/1.0/huggingface-mnist
- Dataset: https://huggingface.co/datasets/ylecun/mnist
- For small image files, the converted the images inside a parquet, so that it is more portable, perhaps not efficient for larger datasets
- Split example: https://github.com/mlcommons/croissant/blob/main/docs/howto/specify-splits.md

In [ ]:
import os

os.chdir("../..")

In [ ]:
import mlcroissant as mlc
from mlcroissant import Dataset
from pathlib import Path
from datetime import datetime
import json
from pelican_data_loader.config import SYSTEM_CONFIG
from pelican_data_loader.utils import get_sha256

In [ ]:
# Instead of using a Git Repo to host data, we will use our development S3 bucket.
# from pelican_data_loader.data import upload_to_s3

# upload_to_s3(
#     file_path=Path("tmp/mnist/test-00000-of-00001.parquet"),
#     object_name="mnist/test.parquet",
# )

# upload_to_s3(
#     file_path=Path("tmp/mnist/train-00000-of-00001.parquet"),
#     object_name="mnist/train.parquet",
# )

In [ ]:
# This is the official metadata for the MNIST dataset in the Croissant format
# For reference
# import httpx

# response = httpx.get(
#     "https://raw.githubusercontent.com/mlcommons/croissant/refs/heads/main/datasets/1.0/huggingface-mnist/metadata.json"
# )
# metadata = response.json()
# Path("tmp/mnist/metadata.json").write_text(json.dumps(metadata, indent=2))

# This is not optimal / usable, because
# 1. Broken sha256, see below s3 FileObject representation
# 2. Even with the correct sha256, mlcroissant does not download the file set correctly from s3 with RecordSet
# 3. It also depends on a very slow way to obtain the train/test split columns dynamically from the file path, which could be useful for larger datasets organized in folder structure. Maybe explore it later.


In [ ]:
# dev_s3 = mlc.FileObject(
#     id="wisc_s3",
#     name="wisc_S3",
#     description="WISC Pelican Data Loader S3 bucket",
#     content_url=f"https://{SYSTEM_CONFIG.s3_url}/",
#     encoding_formats=["https"],  # https is the protocol used for S3
#     sha256="21032ff9ec7c6aed6b2a34399af57090a4195bf099e9846a386c48a1d3260376",  # Placeholder like in official metadata
# )

IMPORTANT LIMITATION on S3 bucket as `FileObject`:

- The official example suggested we should use `FileObject` to represent a S3 bucket
- Currently sha256 is required, see [issue#813](https://github.com/mlcommons/croissant/issues/813)
- But an s3 storage hash will change after something is uploaded to it, so the sha256 will then be changed, there is no way we can 

Example

```py
import httpx
from pelican_data_loader.utils import get_sha256_from_bytes

response = httpx.get(f"https://{SYSTEM_CONFIG.s3_url}/")
get_sha256_from_bytes(response.text.encode("utf-8"))
> '21032ff9ec7c6aed6b2a34399af57090a4195bf099e9846a386c48a1d3260376'
```




In [ ]:
# This represent the parquet file set that contains the MNIST dataset

# BROKEN

# parquet_file_set = mlc.FileSet(
#     id="parquet_files",
#     name="parquet_files",
#     description="Parquet files for MNIST dataset",
#     encoding_formats=[mlc.EncodingFormat.PARQUET],
#     contained_in=[dev_s3.id],
#     # includes=["mnist/train.parquet", "mnist/test.parquet"],
#     includes=["mnist/train.parquet"],
# )

In [ ]:
train_data = mlc.FileObject(
    id="mnist_train",
    name="mnist_train",
    description="MNIST training data in Parquet format",
    encoding_formats=[mlc.EncodingFormat.PARQUET],
    content_url=f"https://{SYSTEM_CONFIG.s3_url}/mnist/train.parquet",
    sha256=get_sha256(Path("tmp/mnist/train-00000-of-00001.parquet")),
)

test_data = mlc.FileObject(
    id="mnist_test",
    name="mnist_test",
    description="MNIST test data in Parquet format",
    encoding_formats=[mlc.EncodingFormat.PARQUET],
    content_url=f"https://{SYSTEM_CONFIG.s3_url}/mnist/test.parquet",
    sha256=get_sha256(Path("tmp/mnist/test-00000-of-00001.parquet")),
)


In [ ]:
def create_record_set(id: str, file_object: mlc.FileObject) -> mlc.RecordSet:
    """Create a RecordSet for the MNIST dataset."""

    image_field = mlc.Field(
        id=f"{id}/image",
        name=f"{id}/image",
        description="Image column in the parquet file",
        data_types=[mlc.DataType.IMAGE_OBJECT],
        source=mlc.Source(
            file_object=file_object.id,
            extract=mlc.Extract(column="image"),
            transforms=[mlc.Transform(json_path="bytes")],
        ),
    )

    label_field = mlc.Field(
        id=f"{id}/label",
        name=f"{id}/label",
        description="Label column in the parquet file",
        data_types=[mlc.DataType.INTEGER],
        source=mlc.Source(file_object=file_object.id, extract=mlc.Extract(column="label")),
    )

    return mlc.RecordSet(
        id=id,
        name=id,
        description=f"{id.title()} record set for MNIST dataset",
        fields=[image_field, label_field],
    )

In [ ]:
mnist_metadata = mlc.Metadata(
    name="mnist",
    description="The MNIST dataset consists of 70,000 28x28 black-and-white images in 10 classes (one for each digits), with 7,000\nimages per class. There are 60,000 training images and 10,000 test images.\n",
    version="1.0.0",
    license=["mit"],
    cite_as="CITE_ME_PLACEHOLDER",
    url="https://huggingface.co/datasets/mnist",
    distribution=[train_data, test_data],
    record_sets=[create_record_set("train", train_data), create_record_set("test", test_data)],
)

In [ ]:
mnist_metadata.issues.report()

In [ ]:
# Patch date

jsonld = mnist_metadata.to_json()
jsonld["datePublished"] = datetime.now().strftime("%Y-%m-%d")
Path("tmp/mnist/replicated_metadata.json").write_text(json.dumps(jsonld, indent=2))


## Loading test

In [ ]:
# Official

# dataset_via_official_metadata = Dataset(jsonld="tmp/mnist/metadata.json")
# official_records = dataset_via_official_metadata.records("default")
# for x in official_records:
#     image = x["default/image"]
#     image.show()
#     break


In [ ]:
# Replicated

dataset_recreated = Dataset(jsonld="tmp/mnist/replicated_metadata.json")
records = dataset_recreated.records("test")


In [ ]:
for i, x in enumerate(records):
    print(x)
    x["test/image"].show()
    if i >= 5:
        break

### Upload metadata to S3

In [ ]:
from pelican_data_loader.data import upload_to_s3

upload_to_s3(
    file_path=Path("tmp/mnist/replicated_metadata.json"),
    object_name="metadata/mnist.json",
)

### Upload to DB

In [ ]:
from pelican_data_loader.db import Dataset, get_session

jsonld = json.loads(Path("tmp/mnist/replicated_metadata.json").read_text())
mnist = Dataset.from_jsonld(jsonld)
mnist.croissant_jsonld_url = f"{SYSTEM_CONFIG.s3_url}/metadata/mnist.json"
mnist

In [ ]:
with get_session() as session:
    session.add(mnist)
    session.commit()

TODO: Cater for multiple files distributions dataset